Importanje potrebnih biblioteka.

In [2]:
import pandas as pd
import scipy.stats as stats

Definiranje raspona generacija i ispravno ocjenjivanje deepfake videa.

In [3]:
# gornji broj je exclusive
gen_z = range(15, 30)
gen_y = range(30, 45)
gen_x = range(45, 65)
gen_babyboom = range(65, 80)
gen_silent = range(80, 100)
not_gen_z = range(30, 100)

# 0 - autentičan, 1 - deepfake (3, 7, 9)
deepfake_correct = [0, 0, 1, 0, 0, 0, 1, 0, 1, 0]
deepfake_strings = [f'deepfake_{i}' for i in range (1, 11)]

Učitavanje dataseta.

In [4]:
dataset = pd.read_excel(r'dataset.xlsx')

"Ispravljanje" deepfake columna u 0 (ako ispitanik nije točno odredio video) ili 1 (ispitanik je točno prepoznao pravi/deepfake video). Dodavanje novog columna 'rezultat' gdje se nalazi ukupan rezultat ispitanika (raspon od 0 do 10).

In [5]:
# odgovori za deepfake ispravljeni u bodove 0 ili 1 ovisno jesu li pogodili
corrected = dataset
for i, ds in enumerate(deepfake_strings):
    corrected[ds] = (corrected[ds] == deepfake_correct[i]).astype(int)

# dodan column rezultat sa zbrojem bodova za svakog ispitanika
corrected['rezultat'] = corrected[list(deepfake_strings)].sum(axis=1)

df = pd.DataFrame(corrected, columns=['ispitanik', 'dob',
                                   'deepfake_1', 'deepfake_2', 'deepfake_3', 'deepfake_4', 'deepfake_5',
                                   'deepfake_6', 'deepfake_7', 'deepfake_8', 'deepfake_9', 'deepfake_10', 'rezultat'], )
print(df)

     ispitanik  dob  deepfake_1  deepfake_2  deepfake_3  deepfake_4  \
0            1   48           1           1           1           0   
1            2   25           1           1           1           1   
2            3   23           1           0           1           0   
3            4   23           1           1           1           1   
4            5   23           1           0           1           1   
..         ...  ...         ...         ...         ...         ...   
97          98   69           1           1           0           1   
98          99   29           1           1           1           1   
99         100   64           1           1           0           1   
100        101   53           0           1           1           1   
101        102   37           1           1           1           1   

     deepfake_5  deepfake_6  deepfake_7  deepfake_8  deepfake_9  deepfake_10  \
0             1           0           1           1           1    

Računanje prosječnog rezultata ispitanika i određivanje granične p vrijednosti.

In [6]:
avg_result = corrected['rezultat'].mean()
print('Prosjek rezultata', avg_result)
p_value = 0.05
print('Kao granična p vrijednost se uzela', p_value)

Prosjek rezultata 7.401960784313726
Kao granična p vrijednost se uzela 0.05


Prvi test:
H0 - Sve generacije jednako dobro raspoznaju deepfake videe
H1 - Generacija Z bolje raspoznaje deepfake u odnosu na ostale generacije

In [11]:
t1h0 = 'Sve generacije jednako dobro raspoznaju deepfake videe'
t1h1 = 'Generacija Z bolje raspoznaje deepfake u odnosu na ostale generacije'
print('H0:', t1h0)
print('H1:', t1h1)
print()

not_gen_z_avg = corrected[corrected['dob'].isin(not_gen_z)]['rezultat'].mean()
print('Prosječni rezultat ispitanika koji nisu dio gen Z generacije:', not_gen_z_avg)
gen_z_data = corrected[corrected['dob'].isin(gen_z)]['rezultat']
print('Prosječni rezultat ispitanika koji su dio gen Z generacije:', gen_z_data.mean())
_, p_value_test = stats.ttest_1samp(gen_z_data, not_gen_z_avg)

print('P vrijednost je', p_value_test)
if p_value_test < p_value:
    print('Odbacuje se H0.')
else:
    print('Nije moguće odbaciti H0.')

H0: Sve generacije jednako dobro raspoznaju deepfake videe
H1: Generacija Z bolje raspoznaje deepfake u odnosu na ostale generacije

Prosječni rezultat ispitanika koji nisu dio gen Z generacije: 7.083333333333333
Prosječni rezultat ispitanika koji su dio gen Z generacije: 7.685185185185185
P vrijednost je 0.014324984472095098
Odbacuje se H0.


Drugi test:
H0 - Svi ispitanici podjednako dobro raspoznaju deepfake videe neovisno o tome koliko prate društvene mreže
H1 - Ispitanici koji društvene mreže prate barem jednom dnevno bolje prepoznaju deepfake videe

In [12]:
t2h0 = 'Svi ispitanici podjednako dobro raspoznaju deepfake videe neovisno o tome koliko prate društvene mreže'
t2h1 = 'Ispitanici koji društvene mreže prate barem jednom dnevno bolje prepoznaju deepfake videe'
print('H0:', t2h0)
print('H1:', t2h1)
print()

not_social_media_avg = corrected[corrected['cesto_mreze'] > 1]['rezultat'].mean()
print('Prosječni rezultat ispitanika koji rijetko prate društvene mreže:', not_social_media_avg)
social_media_data = corrected[corrected['cesto_mreze'] == 1]['rezultat']
print('Prosječni rezultat ispitanika koji često prate društvene mreže:', social_media_data.mean())
_, p_value_test = stats.ttest_1samp(social_media_data, not_social_media_avg)

print('P vrijednost je', p_value_test)
if p_value_test < p_value:
    print('Odbacuje se H0.')
else:
    print('Nije moguće odbaciti H0.')

H0: Svi ispitanici podjednako dobro raspoznaju deepfake videe neovisno o tome koliko prate društvene mreže
H1: Ispitanici koji društvene mreže prate barem jednom dnevno bolje prepoznaju deepfake videe

Prosječni rezultat ispitanika koji rijetko prate društvene mreže: 6.7368421052631575
Prosječni rezultat ispitanika koji često prate društvene mreže: 7.554216867469879
P vrijednost je 2.0550580600501343e-05
Odbacuje se H0.


Treći test:
H0 - Svi ispitanici podjednako dobro raspoznaju deepfake videe neovisno o tome koliko imaju povjerenja u medije
H1 - Ispitanici koji uopće nemaju povjerenja u medije ne raspoznaju dobro deepfake videe

In [14]:
t3h0 = 'Svi ispitanici podjednako dobro raspoznaju deepfake videe neovisno o tome koliko imaju povjerenja u medije'
t3h1 = 'Ispitanici koji uopće nemaju povjerenja u medije ne raspozaju dobro deepfake videe'
print('H0:', t3h0)
print('H1:', t3h1)
print()

# povjerenje u tv
trust_tv_avg = corrected[corrected['povjerenje_tv'] <= 3]['rezultat'].mean()
print('Prosječni rezultat ispitanika koji vjeruju televiziji:', trust_tv_avg)
not_trust_tv_data = corrected[corrected['povjerenje_tv'] > 3]['rezultat']
print('Prosječni rezultat ispitanika koji ne vjeruju televiziji:', not_trust_tv_data.mean())
_, p_value_test = stats.ttest_1samp(not_trust_tv_data, trust_tv_avg)

print('P vrijednost je', p_value_test)
if p_value_test < p_value:
    print('Odbacuje se H0.')
else:
    print('Nije moguće odbaciti H0.')
print()

# povjerenje u radio
trust_radio_avg = corrected[corrected['povjerenje_radio'] <= 3]['rezultat'].mean()
print('Prosječni rezultat ispitanika koji vjeruju radiju:', trust_radio_avg)
not_trust_radio_data = corrected[corrected['povjerenje_radio'] > 3]['rezultat']
print('Prosječni rezultat ispitanika koji ne vjeruju radiju:', not_trust_radio_data.mean())
_, p_value_test = stats.ttest_1samp(not_trust_radio_data, trust_radio_avg)

print('P vrijednost je', p_value_test)
if p_value_test < p_value:
    print('Odbacuje se H0.')
else:
    print('Nije moguće odbaciti H0.')
print()

#povjerenje u novine
trust_news_avg = corrected[corrected['povjerenje_novine'] <= 3]['rezultat'].mean()
print('Prosječni rezultat ispitanika koji vjeruju novinama:', trust_news_avg)
not_trust_news_data = corrected[corrected['povjerenje_novine'] > 3]['rezultat']
print('Prosječni rezultat ispitanika koji ne vjeruju novinama:', not_trust_news_data.mean())
_, p_value_test = stats.ttest_1samp(not_trust_news_data, trust_news_avg)

print('P vrijednost je', p_value_test)
if p_value_test < p_value:
    print('Odbacuje se H0.')
else:
    print('Nije moguće odbaciti H0.')
print()

#povjerenje u portale
trust_portal_avg = corrected[corrected['povjerenje_portali'] <= 3]['rezultat'].mean()
print('Prosječni rezultat ispitanika koji vjeruju portalima:', trust_portal_avg)
not_trust_portal_data = corrected[corrected['povjerenje_portali'] > 3]['rezultat']
print('Prosječni rezultat ispitanika koji ne vjeruju portalima:', not_trust_portal_data.mean())
_, p_value_test = stats.ttest_1samp(not_trust_portal_data, trust_portal_avg)

print('P vrijednost je', p_value_test)
if p_value_test < p_value:
    print('Odbacuje se H0.')
else:
    print('Nije moguće odbaciti H0.')
print()

# povjerenje u društvene mreže
trust_sm_avg = corrected[corrected['povjerenje_mreze'] <= 3]['rezultat'].mean()
print('Prosječni rezultat ispitanika koji vjeruju društvenim mrežama:', trust_sm_avg)
not_trust_sm_data = corrected[corrected['povjerenje_mreze'] > 3]['rezultat']
print('Prosječni rezultat ispitanika koji ne vjeruju društvenim mrežama:', not_trust_sm_data.mean())
_, p_value_test = stats.ttest_1samp(not_trust_sm_data, trust_sm_avg)

print('P vrijednost je', p_value_test)
if p_value_test < p_value:
    print('Odbacuje se H0.')
else:
    print('Nije moguće odbaciti H0.')
print()

H0: Svi ispitanici podjednako dobro raspoznaju deepfake videe neovisno o tome koliko imaju povjerenja u medije
H1: Ispitanici koji uopće nemaju povjerenja u medije ne raspozaju dobro deepfake videe

Prosječni rezultat ispitanika koji vjeruju televiziji: 7.4605263157894735
Prosječni rezultat ispitanika koji ne vjeruju televiziji: 7.230769230769231
P vrijednost je 0.5251477006498033
Nije moguće odbaciti H0.

Prosječni rezultat ispitanika koji vjeruju radiju: 7.512820512820513
Prosječni rezultat ispitanika koji ne vjeruju radiju: 7.041666666666667
P vrijednost je 0.26271987675661923
Nije moguće odbaciti H0.

Prosječni rezultat ispitanika koji vjeruju novinama: 7.506329113924051
Prosječni rezultat ispitanika koji ne vjeruju novinama: 7.043478260869565
P vrijednost je 0.24794753970701763
Nije moguće odbaciti H0.

Prosječni rezultat ispitanika koji vjeruju portalima: 7.491803278688525
Prosječni rezultat ispitanika koji ne vjeruju portalima: 7.2682926829268295
P vrijednost je 0.45516834085782